In [1]:
from scipy.ndimage import gaussian_laplace
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math

In [2]:
def read_video(video):
    video = cv2.VideoCapture(video)
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_fps = int(video.get(cv2.CAP_PROP_FPS))
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Exit if video not opened.
    if not video.isOpened():
        print("Could not open video")
        sys.exit()

    return (video, frame_width, frame_height, video_fps, frame_count)

In [3]:
def moving_average(curve, radius):
    window_size = 2 * radius + 1
    # Define the filter
    f = np.ones(window_size)/window_size
    
    # Add padding to the boundaries
    curve_pad = np.lib.pad(curve, (radius, radius), 'edge')
    
    # Apply convolution
    curve_smoothed = np.convolve(curve_pad, f, mode='same')
    
    # Remove padding
    curve_smoothed = curve_smoothed[radius:-radius]
    
    # return smoothed curve
    return curve_smoothed

In [4]:
def smooth(trajectory):
    smoothed_trajectory = np.copy(trajectory)
    
    # Filter the x, y and angle curves
    for i in range(3):
        smoothed_trajectory[:,i] = moving_average(trajectory[:,i], radius=10)

    return smoothed_trajectory

In [5]:
def laplacian_of_gaussian(trajectory):
    smoothed_trajectory = np.copy(trajectory)
    
    # Filter the x, y and angle curves by first using Gaussian blur to smooth the frame
    # and then using a Laplacian transform to apply convolution
    for i in range(3):
        smoothed_trajectory[:,i] = gaussian_laplace(trajectory[:,i], sigma=10)

    return smoothed_trajectory

In [6]:
def fixBorder(frame):
    s = frame.shape
    
    # Scale the image 10% without moving the center
    T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.1)
    frame = cv2.warpAffine(frame, T, (s[1], s[0]))

    return frame

In [7]:
# compute peak-signal-to-noise ratio between frames
def calculate_psnr(video, frame_count):
    psnr = np.zeros(frame_count-1, np.float32)
    
    # Read first frame
    _, prev_frame = video.read()
    
    for i in range(frame_count-2):
        re, frame = video.read()
        if re == False:
            break
        
        mse = np.mean((np.array(prev_frame, dtype=np.float32) - np.array(frame, dtype=np.float32)) ** 2)
        
        if mse == 0:
            psnr[i] = 100
        else:
            psnr[i] = 20 * np.log10(255 / (np.sqrt(mse)))

        # Move to next frame
        prev_frame = frame
    
    return psnr

In [8]:
def compute_motion_estimation(video, frame_count, frame_width, frame_height):
    transforms = np.zeros((frame_count-1, 3), np.float32)
    
    # Read first frame
    _, prev_frame = video.read()

    # Convert frame to grayscale
    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    for i in range(frame_count-2):
        re, frame = video.read()
        if re == False:
            break

        # convert current frame to grayscale
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect feature points in previous frame
        prev_pts = cv2.goodFeaturesToTrack(prev_frame,
                                           maxCorners=200,
                                           qualityLevel=0.01,
                                           minDistance=30,
                                           blockSize=3)

        # Calculate optical flow (i.e. track feature points)
        curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_frame, frame, prev_pts, None)

        # Sanity check
        assert prev_pts.shape == curr_pts.shape

        # Filter only valid points
        idx = np.where(status==1)[0]
        prev_pts = prev_pts[idx]
        curr_pts = curr_pts[idx]

        # Find transformation matrix
        m = cv2.estimateAffinePartial2D(prev_pts, curr_pts)
        
        # Extract translation
        dx = m[0][0,2]
        dy = m[0][1,2]

        # Extract rotation angle
        da = np.arctan2(m[0][1,0], m[0][0,0])

        # Store transformation
        transforms[i] = [dx,dy,da]

        # Move to next frame
        prev_frame = frame
    
    return transforms

In [9]:
def compute_smooth_motion(transforms, gaussian=False):
    # Compute trajectory using cumulative sum of transformations
    trajectory = np.cumsum(transforms, axis=0)
    
    # Calculate smoothed directory
    if gaussian is True:
        smoothed_trajectory = laplacian_of_gaussian(trajectory)
    else:
        smoothed_trajectory = smooth(trajectory)

    # Calculate difference in smoothed_trajectory and trajectory
    difference = smoothed_trajectory - trajectory

    # Calculate newer transformation array
    transforms_smooth = transforms + difference
    
    return transforms_smooth

In [10]:
def apply_new_transforms(video, frame_count, frame_width, frame_height, out, new_transforms, gaussian=False):
    # Reset stream to first frame
    video.set(cv2.CAP_PROP_POS_FRAMES, 0)

    for i in range(frame_count-2):
        ret, frame = video.read()

        if ret == False:
            break
            
        if gaussian is False:
            # Extract transformations from the new transformation array
            dx = new_transforms[i,0]
            dy = new_transforms[i,1]
            da = new_transforms[i,2]

            # Reconstruct transformation matrix accordingly to new values
            m = np.zeros((2,3), np.float32)
            m[0,0] = np.cos(da)
            m[0,1] = -np.sin(da)
            m[1,0] = np.sin(da)
            m[1,1] = np.cos(da)
            m[0,2] = dx
            m[1,2] = dy

            # Apply affine wrapping to the given frame
            frame = cv2.warpAffine(frame, m, (frame_width,frame_height))
        
        # Fix border artifacts
        frame_stabilized = fixBorder(frame)
        
        # Write new video to output
        out.write(frame_stabilized)

        # Write the frame to the file
        frame_out = cv2.hconcat([frame, frame_stabilized])

        # If the image is too big, resize it.
        if(frame_out.shape[1] > 1920):
            frame_out = cv2.resize(frame_out, (int(frame_out.shape[1]/3), int(frame_out.shape[0]/3)));
            
        cv2.imshow("Before and After", frame_out)
        cv2.waitKey(10)

In [11]:
# collect input video object and characteristics
video, frame_width, frame_height, video_fps, frame_count = read_video("zilker_park.mp4") # provided in GitHub
#video, frame_width, frame_height, video_fps, frame_count = read_video("Videos/Clip_12.mov") # from dataset

In [12]:
# get peak-signal-to-noise ratio for original frames
original_psnr = calculate_psnr(video, frame_count)
original_psnr_avg = np.average(original_psnr)
print("The average peak-signal-to-noise ratio for original video frames is: {}".format(original_psnr_avg))

The average peak-signal-to-noise ratio for original video frames is: 32.81125259399414


In [13]:
# Reset stream to first frame
video.set(cv2.CAP_PROP_POS_FRAMES, 0)

True

In [15]:
# compute optical flow for video
transforms = compute_motion_estimation(video, frame_count, frame_width, frame_height)

In [15]:
# calculate new transforms with moving window average filter
new_transforms = compute_smooth_motion(transforms)

In [16]:
# Set up output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('video_out_no_gauss.avi', fourcc, video_fps, (frame_width, frame_height))

# apply new transforms to video file
apply_new_transforms(video, frame_count, frame_width, frame_height, out, new_transforms, gaussian=False)

video.release()
out.release()
cv2.destroyAllWindows()

In [17]:
# get peak-signal-to-noise ratio for new frames
output_vid, frame_width, frame_height, video_fps, frame_count = read_video("video_out_no_gauss.avi")

new_psnr = calculate_psnr(output_vid, frame_count)
new_psnr_avg = np.average(new_psnr)
print("The average peak-signal-to-noise ratio for transformed video frames is: {}".format(new_psnr_avg))

output_vid.release()
cv2.destroyAllWindows()

The average peak-signal-to-noise ratio for transformed video frames is: 34.94142150878906


In [16]:
# calculate new transforms with laplacian of gaussian filter
new_transforms = compute_smooth_motion(transforms, gaussian=True)

In [17]:
#Set up output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('video_out_gauss.avi', fourcc, video_fps, (frame_width, frame_height))

# apply new transforms to video file
apply_new_transforms(video, frame_count, frame_width, frame_height, out, new_transforms, gaussian=True)

video.release()
out.release()
cv2.destroyAllWindows()

In [18]:
# get peak-signal-to-noise ratio for new frames
output_vid, frame_width, frame_height, video_fps, frame_count = read_video("video_out_gauss.avi")

new_psnr = calculate_psnr(output_vid, frame_count)
new_psnr_avg = np.average(new_psnr)
print("The average peak-signal-to-noise ratio for transformed video frames is: {}".format(new_psnr_avg))

output_vid.release()
cv2.destroyAllWindows()

The average peak-signal-to-noise ratio for transformed video frames is: 33.38200759887695
